In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Načtení modelu po fine-tuningu a vstupních dat
lora_path = "./r8_mine_gwen" 
csv_file = "CSV-Hashed.csv"

# Načtení konfigurace LoRA modelu
peft_config = PeftConfig.from_pretrained(lora_path)

# Načtení base modelu Qwen
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Připojení LoRA adaptéru
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()

# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True
)

# Načtení CSV jako text
df = pd.read_csv(csv_file)
csv_text = df.to_csv(index=False)

# Tvorba promptu pro Gwen
qwen_prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "You are a data analyst who always returns frequent itemsets in a clean format (no code)."},
        {"role": "user", "content": f"I will give you CSV and your job is to find frequent itemsets of maxlenght 5 with support 0.75. Do not use any code, it is your job to do!<csv>{csv_text}</csv>"}
    ],
    tokenize=False,
    add_generation_prompt=True
)

# Tokenizace vstupu
inputs = tokenizer(qwen_prompt, return_tensors="pt").to(model.device)

# Generování výstupu
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10000,  # Zvolena vysoká hodnota pro možnost generování rozsáhlých odpovědí
        temperature=1, # nebo 0001, nebo 0.5
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
# Dekódování výstupu modelu do normálního textu
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Uložení výsledku
with open("model_output.txt", "w", encoding="utf-8") as f:
    f.write(decoded_output)

print("Výstup byl uložen do 'model_output.txt'")


/opt/tljh/user/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-04-18 20:28:37.131429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 20:28:37.142162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745000917.154145 2677897 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000

✅ Výstup byl uložen do 'model_output.txt'
